In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader


In [3]:
# service options
delimiter = '/'

## configuration options
data_path = '/home/cosmonaut/dev/pic/picopic/'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

timestamp=2.85e-10
cmap='terrain'
image_interpolation = 'nearest'

autoselect = True
show_grid=False

dim_size = 0.01

dim = 'radius'

guess_number_ticks=20
ylim = None

cmp = 'E/r'
specie = None # simply init with none. Replace with required specie for specied grids
probe_path = None

if specie:
    probe_path = cmp + delimiter + specie
else:
    probe_path = cmp

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

r_scale = (size[2] - size[0]) / reader.meta.geometry_grid[0]
z_scale = (size[3] - size[1]) / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
clim = [-clim_estimation, clim_estimation]

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'

plot_name = r"$\mathbf{{Mesh\enspace Color\enspace Map\enspace for\enspace {x}}}$".format(x=probe_path)

In [4]:
dim_mesh_number = 0

if dim == 'radius':
    dim_mesh_number = reader.meta.get_row_by_radius(dim_size)
elif dim == 'longitude':
    dim_mesh_number = reader.meta.get_col_by_longitude(dim_size)
else:
    raise ValueError("Dimension name", dim, "is wrong")

# get data
data = []

for probe in reader.meta.probes:
    frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if dim == 'radius':
        if probe.shape == 'row' and probe.size[2] == dim_mesh_number and probe.component == cmp:
            data = reader.row(probe_path, probe.size, frame)
    elif dim == 'longitude':
        if probe.shape == 'col' and probe.size[3] == dim_mesh_number and probe.component == cmp:
            data = reader.col(probe_path, probe.size, frame)

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in reader.meta.probes:
        frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if ( probe.shape == 'rec'
            and probe.component == cmp
            and probe.size[1] == 0
            and probe.size[0] <= dim_mesh_number
            and probe.size[3] == reader.meta.geometry_grid[1]
            and probe.size[2] >= dim_mesh_number ):
            if dim == 'radius':
                data = reader.row_rec(probe_path, dim_mesh_number, probe.size, frame)
            elif dim == 'longitude':
                data = reader.col_rec(probe_path, dim_mesh_number, probe.size, frame)

Loading data set E/r/rec:0-64_0-256/57...done


In [5]:
data_len = np.linspace(size[0], size[2], len(data))

# define plot builder
plot = PlotBuilder(0, 0,
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   
                   x_ticklabel_end=reader.meta.geometry_size[1] * z_scale,
                   y_ticklabel_end=reader.meta.geometry_size[0] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', image_interpolation=image_interpolation, 
                   guess_number_ticks = guess_number_ticks)

In [6]:
# set y-limits
if ylim is not None:
    plot.set_ylim(ylim)

# add subplots
sub_plot = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add data
sub_plot.plot(data_len, data)

plot.show()